In [1]:
!pip install praw
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 9.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import praw
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import zstandard as zstd
from datetime import datetime


### Methodology 1. Reddit praw api

In [2]:
### put your personal info 
reddit = praw.Reddit(client_id='', \
                     client_secret='', \
                     user_agent='')

In [ ]:
posts = []
ml_subreddit = reddit.subreddit('depression')
for post in ml_subreddit.new(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
posts

In [8]:
posts = []
ml_subreddit = reddit.subreddit('depression')
for post in ml_subreddit.hot(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [9]:
posts

,title,score,id,subreddit,url,num_comments,body,created
0,Our most-broken and least-understood rules is ...,2357,doqwow,depression,https://www.reddit.com/r/depression/comments/d...,177,We understand that most people who reply immed...,1.572361e+09
1,"Regular check-in post, with information about ...",66,16kqyd7,depression,https://www.reddit.com/r/depression/comments/1...,395,Welcome to /r/depression's check-in post - a p...,1.694923e+09
2,This loneliness is getting pretty bad,63,1bm7xyn,depression,https://www.reddit.com/r/depression/comments/1...,18,The drugs don't even help. Why are ppl so mean...,1.711241e+09
3,Why is the younger generation depressed.,16,1bmdi9s,depression,https://www.reddit.com/r/depression/comments/1...,14,1. Cost of living being high.\n2. Toxic work c...,1.711259e+09
4,"I'm 15, suicidal and addicted to weed.",77,1bm2dd1,depression,https://www.reddit.com/r/depression/comments/1...,39,"About 5 months ago, I moved into a group home ...",1.711227e+09
...,...,...,...,...,...,...,...,...
843,I need help,0,1bjjxmj,depression,https://www.reddit.com/r/depression/comments/1...,0,Me and my gf were together for three years but...,1.710958e+09
844,Really struggling,2,1bjegpr,depression,https://www.reddit.com/r/depression/comments/1...,0,I have a complex medical disease(MCAS/POTS). I...,1.710944e+09
845,Depression makes you self-sabotage friendships,2,1bjecph,depression,https://www.reddit.com/r/depression/comments/1...,0,A lot of people seem to avoid others who are d...,1.710944e+09
846,Depressed since childhood,0,1bjjq1n,depression,https://www.reddit.com/r/depression/comments/1...,0,I’ve been trying to find the person I was befo...,1.710957e+09


In [10]:
posts.title[0]

'Our most-broken and least-understood rules is "helpers may not invite private contact as a first resort", so we\'ve made a new wiki to explain it'

In [11]:
posts.body[0]

'We understand that most people who reply immediately to an OP with an invitation to talk privately  mean only to help, but this type of response usually leads to either disappointment or disaster.  it usually works out quite differently here than when you say "PM me anytime" in a casual social context.  \n\nWe have huge admiration and appreciation for the goodwill and good citizenship of so many of you who support others here and flag inappropriate content - even more so because we know that so many of you are struggling yourselves.  We\'re hard at work behind the scenes on more information and resources to make it easier to give and get quality help here - this is just a small start.  \n\nOur new wiki page explains in detail why it\'s much better to respond in public comments, at least until you\'ve gotten to know someone.  It will be maintained at /r/depression/wiki/private_contact, and the full text of the current version is below.\n\n*****\n\n###Summary###\n\n**Anyone who, while a

### Methodology 2. Reddit archive data dump (https://the-eye.eu/redarcs/) 

In [4]:
ls

depression_submissions.zst             reddit_data_preprocessing_happy.ipynb
happy_submissions.zst                  reddit_data_preprocessing_v1.ipynb


In [ ]:
## step1. load downloaded submission reddit dump 

In [39]:
happy_submissions =\
    pd.read_json('happy_submissions.zst',compression=dict(method='zstd', max_window_size=2147483648),\
    lines = True, nrows = 2000000)

In [40]:
# convert the timestamp to utc
happy_submissions['created_utc'] =\
    happy_submissions['created_utc'].apply(lambda x : datetime.utcfromtimestamp(x))

In [41]:
## step2. download the file into parquet file to speed up data load later stage

In [42]:
happy_submissions.to_parquet('happy_submissions.parquet', engine='pyarrow')

In [43]:
## step3. read as parquet file 

In [44]:
happy_submissions = pd.read_parquet('happy_submissions.parquet',engine='pyarrow')
happy_submissions

,archived,author,author_flair_background_color,author_flair_css_class,author_flair_text,author_flair_text_color,brand_safe,can_gild,contest_mode,created_utc,distinguished,domain,edited,gilded,hidden,hide_score,id,is_crosspostable,is_reddit_media_domain,is_self,is_video,link_flair_css_class,link_flair_richtext,link_flair_text,link_flair_text_color,link_flair_type,locked,media,media_embed,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,retrieved_on,rte_mode,score,secure_media,secure_media_embed,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,url,whitelist_status,author_flair_richtext,author_flair_type,post_hint,preview,author_cakeday,is_original_content,downs,num_reports,banned_by,name,likes,clicked,saved,ups,approved_by,selftext_html,created,mod_reports,user_reports,report_reasons,from_kind,from_id,quarantine,from,view_count,approved_at_utc,banned_at_utc,can_mod_post,pinned,crosspost_parent,crosspost_parent_list,mod_note,mod_reason_by,mod_reason_title,subreddit_subscribers,media_metadata,author_flair_template_id,category,content_categories,media_only,post_categories,pwls,removal_reason,visited,wls,link_flair_background_color,link_flair_template_id,previous_visits,author_created_utc,author_fullname,is_meta,author_patreon_flair,gildings,is_robot_indexable,all_awardings,total_awards_received,allow_live_comments,discussion_type,awarders,removed_by,removed_by_category,author_premium,treatment_tags,poll_data,upvote_ratio,is_created_from_ads_ui,retrieved_utc,top_awarded_type,url_overridden_by_dest,gallery_data,is_gallery,call_to_action
0,1.0,[deleted],,None,None,dark,1.0,0.0,0.0,2008-01-26 02:17:12,None,hosted.ap.org,0,0.0,0.0,0.0,66nlf,1.0,0.0,False,0.0,None,[],None,dark,text,0.0,None,"{'content': None, 'height': None, 'media_domai...",1.0,0,0.0,False,all_ads,/r/happy/comments/66nlf/its_good_to_know_there...,1.522683e+09,markdown,8,None,"{'content': None, 'height': None, 'media_domai...",,1.0,0.0,0.0,happy,t5_2qh5i,r/happy,public,NaN,default,NaN,NaN,It's good to know there are some honest people...,http://hosted.ap.org/dynamic/stories/M/MISSING...,all_ads,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,NaN,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,NaN,None,None,NaN,NaN,NaN,None,None,None,NaN,None
1,1.0,hitsman,None,None,None,None,1.0,1.0,0.0,2008-01-26 04:05:36,None,youtube.com,0,0.0,0.0,0.0,66nsm,1.0,0.0,False,0.0,None,[],None,dark,text,0.0,None,"{'content': None, 'height': None, 'media_domai...",0.0,1,0.0,False,all_ads,/r/happy/comments/66nsm/this_baby_seems_to_thi...,1.522683e+09,markdown,5,None,"{'content': None, 'height': None, 'media_domai...",,1.0,0.0,0.0,happy,t5_2qh5i,r/happy,public,NaN,default,NaN,NaN,This baby seems to think Darth Vader is a joke...,http://www.youtube.com/watch?v=rfpK3H5e0S4,all_ads,[],text,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,NaN,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,NaN,None,None,NaN,NaN,NaN,None,None,None,NaN,None
2,1.0,[deleted],,None,None,dark,1.0,0.0,0.0,2008-01-26 21:57:24,None,news.yahoo.com,0,0.0,0.0,0.0,66pr5,0.0,0.0,False,0.0,None,[],None,dark,text,0.0,None,"{'content': None, 'height': None, 'media_domai...",1.0,0,0.0,False,all_ads,/r/happy/comments/66pr5/ap_a_camera_is_found_i...,1.522683e+09,markdown,0,None,"{'content': None, 'height': None, 'media_domai...",[deleted],1.0,0.0,0.0,happy,t5_2qh5i,r/happy,public,NaN,default,NaN,NaN,AP\nA Camera is found in the back of a Taxi. T...,http://news.yahoo.com/s/ap/20080125/ap_on_hi_t...,all_ads,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN

In [45]:
## step4. feature selection : select only necessary columns

In [46]:
happy_submissions =\
    happy_submissions[['subreddit','author','over_18','link_flair_text','title','selftext','url','created_utc']]
happy_submissions

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,happy,[deleted],False,None,It's good to know there are some honest people...,,http://hosted.ap.org/dynamic/stories/M/MISSING...,2008-01-26 02:17:12
1,happy,hitsman,False,None,This baby seems to think Darth Vader is a joke...,,http://www.youtube.com/watch?v=rfpK3H5e0S4,2008-01-26 04:05:36
2,happy,[deleted],False,None,AP\nA Camera is found in the back of a Taxi. T...,[deleted],http://news.yahoo.com/s/ap/20080125/ap_on_hi_t...,2008-01-26 21:57:24
3,happy,[deleted],False,None,A camera is left in the back of a taxi. The on...,[deleted],http://news.yahoo.com/s/ap/20080125/ap_on_hi_t...,2008-01-26 21:58:04
4,happy,glmory,False,None,Hawaii on a Dime,,http://travel.nytimes.com/2008/01/27/travel/27...,2008-01-27 15:12:28
...,...,...,...,...,...,...,...,...
100029,happy,[deleted],False,None,90s sleepovers nostalgia,[removed],,2022-12-31 22:01:33
100030,happy,RainbowBlasts,False,None,HAPPY NEW YEAR REDDIT,,https://i.redd.it/xzd4nnz7zc9a1.gif,2022-12-31 23:22:12
100031,happy,not-linda,False,None,"Three weeks ago, my 55 yo husband had a heart ...",,https://www.reddit.com/r/happy/comments/1006o1...,2022-12-31 23:36:23
100032,happy,[deleted],False,None,"Have A Wonderful, Awesome, Great, Incredible N...",[deleted],,2022-12-31 23:45:47


In [48]:
## step5. basic pre-processing 

In [47]:
# remove blank body posts
happy_submissions = happy_submissions[happy_submissions['selftext'].apply(lambda x : len(x)!=0)]
happy_submissions

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
2,happy,[deleted],False,None,AP\nA Camera is found in the back of a Taxi. T...,[deleted],http://news.yahoo.com/s/ap/20080125/ap_on_hi_t...,2008-01-26 21:57:24
3,happy,[deleted],False,None,A camera is left in the back of a taxi. The on...,[deleted],http://news.yahoo.com/s/ap/20080125/ap_on_hi_t...,2008-01-26 21:58:04
80,happy,[deleted],False,None,Can Education Research Save Us? An Upbeat View,[deleted],http://www.washingtonpost.com/wp-dyn/content/a...,2008-02-04 23:45:50
102,happy,[deleted],False,None,Its Pitch Dark,[deleted],http://www.youtube.com/watch?v=eA6G5LvozlE,2008-02-07 03:42:21
104,happy,[deleted],False,None,Cats In Space!!!! [video],[deleted],http://www.youtube.com/watch?v=fUIokQ36rbA,2008-02-07 03:45:22
...,...,...,...,...,...,...,...,...
100020,happy,SmokePeltwarriorcats,False,None,this title system is weird and and I just want...,"\nI came from a horribly abusive household, my...",https://www.reddit.com/r/happy/comments/zzwxld...,2022-12-31 15:49:35
100026,happy,iamdominicparkhurst,False,None,I want to sign up to a site where I get positi...,"Hi, \n\nI want to feel better. Where can I hav...",https://www.reddit.com/r/happy/comments/1001wh...,2022-12-31 19:40:45
100028,happy,iamdominicparkhurst,False,None,What makes you happy on a rainy day,[removed],https://www.reddit.com/r/happy/comments/1003sx...,2022-12-31 21:12:05
100029,happy,[deleted],False,None,90s sleepovers nostalgia,[removed],,2022-12-31 22:01:33


In [49]:
## opt out '[deleted]','[removed]' only selftext 
opt_out_list = ['[deleted]','[removed]']
happy_submissions[happy_submissions['selftext'].isin(opt_out_list)]

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
2,happy,[deleted],False,None,AP\nA Camera is found in the back of a Taxi. T...,[deleted],http://news.yahoo.com/s/ap/20080125/ap_on_hi_t...,2008-01-26 21:57:24
3,happy,[deleted],False,None,A camera is left in the back of a taxi. The on...,[deleted],http://news.yahoo.com/s/ap/20080125/ap_on_hi_t...,2008-01-26 21:58:04
80,happy,[deleted],False,None,Can Education Research Save Us? An Upbeat View,[deleted],http://www.washingtonpost.com/wp-dyn/content/a...,2008-02-04 23:45:50
102,happy,[deleted],False,None,Its Pitch Dark,[deleted],http://www.youtube.com/watch?v=eA6G5LvozlE,2008-02-07 03:42:21
104,happy,[deleted],False,None,Cats In Space!!!! [video],[deleted],http://www.youtube.com/watch?v=fUIokQ36rbA,2008-02-07 03:45:22
...,...,...,...,...,...,...,...,...
100018,happy,SmokePeltwarriorcats,False,None,my boyfriend is a dream❤️,[removed],https://www.reddit.com/r/happy/comments/zzwt80...,2022-12-31 15:43:38
100019,happy,SmokePeltwarriorcats,False,None,"I came from an abusive home, but my partner re...",[removed],https://www.reddit.com/r/happy/comments/zzwumc...,2022-12-31 15:45:34
100028,happy,iamdominicparkhurst,False,None,What makes you happy on a rainy day,[removed],https://www.reddit.com/r/happy/comments/1003sx...,2022-12-31 21:12:05
100029,happy,[deleted],False,None,90s sleepovers nostalgia,[removed],,2022-12-31 22:01:33


In [50]:
happy_submissions = happy_submissions[~happy_submissions['selftext'].isin(opt_out_list)]
print(len(happy_submissions))
happy_submissions

18231


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
2847,happy,thenewhustle,False,None,How To Be Happy(er) - Sionara Slumps,"Understand happiness is not the destination, i...",https://www.reddit.com/r/happy/comments/8wz2g/...,2009-06-30 14:08:37
3017,happy,calantorntain,False,None,"Hey Reddit, just so you know, I'm happy right ...",I'm usually just content. It isn't unpleasant...,https://www.reddit.com/r/happy/comments/94x28/...,2009-07-27 12:06:03
3035,happy,anonysumo,False,None,The habitual noise polluter who's been driving...,There's a car wash about 300 feet from my hous...,https://www.reddit.com/r/happy/comments/965gu/...,2009-07-30 23:05:57
3102,happy,anonymousgangster,False,None,I received this in my email and it's pretty pr...,READ THIS SLOWLY... IT'S PRETTY PROFOUND.\n\nT...,https://www.reddit.com/r/happy/comments/98vdi/...,2009-08-09 03:18:07
3110,happy,marquis_of_chaos,False,None,Meet Iris the Bengal cat,Iris the Bengal cat [lives on a boat](http://w...,https://www.reddit.com/r/happy/comments/99amy/...,2009-08-10 17:55:42
...,...,...,...,...,...,...,...,...
99995,happy,just_amanda_,False,None,I’m living a life that seemed impossible five ...,At that point I was on the verge of dropping o...,https://www.reddit.com/r/happy/comments/zysrie...,2022-12-30 06:02:02
100007,happy,designsims,False,None,I'm curious: What was the best singular experi...,"In my search for happiness, I'm wondering what...",https://www.reddit.com/r/happy/comments/zz88nn...,2022-12-30 18:51:59
100015,happy,sharmakiran96,False,None,Celebrate endings—for they precede new beginni...,&amp;#x200B;\n\nhttps://preview.redd.it/b5fvfn...,https://www.reddit.com/r/happy/comments/zzt44v...,2022-12-31 12:30:16
100020,happy,SmokePeltwarriorcats,False,None,this title system is weird and and I just want...,"\nI came from a horribly abusive household, my...",https://www.reddit.com/r/happy/comments/zzwxld...,2022-12-31 15:49:35


In [51]:
# step6. final data quality check 

In [55]:
print(f'Author : {happy_submissions.iloc[[0],:].author.tolist()[0]}')
print(f'Subreddit : {happy_submissions.iloc[[0],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {happy_submissions.iloc[[0],:].over_18.tolist()[0]}')
print(f'Posted : {happy_submissions.iloc[[0],:].created_utc.tolist()[0]}')
print()
print(f'Title : {happy_submissions.iloc[[0],:].title.tolist()[0]}')
print(f'Body : {happy_submissions.iloc[[0],:].selftext.tolist()[0]}')
print()
print()
print()
print(f'Author : {happy_submissions.iloc[[1],:].author.tolist()[0]}')
print(f'Subreddit : {happy_submissions.iloc[[1],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {happy_submissions.iloc[[1],:].over_18.tolist()[0]}')
print(f'Posted : {happy_submissions.iloc[[1],:].created_utc.tolist()[0]}')
print()
print(f'Title : {happy_submissions.iloc[[1],:].title.tolist()[0]}')
print(f'Body : {happy_submissions.iloc[[1],:].selftext.tolist()[0]}')


Author : thenewhustle
Subreddit : happy
Teenager(True/False) : False
Posted : 2009-06-30 14:08:37

Title : How To Be Happy(er) - Sionara Slumps
Body : Understand happiness is not the destination, it's in the journey!  Inspiration for our generation. 



Author : calantorntain
Subreddit : happy
Teenager(True/False) : False
Posted : 2009-07-27 12:06:03

Title : Hey Reddit, just so you know, I'm happy right now.
Body : I'm usually just content.  It isn't unpleasant, but it's just... contentment.

But I was driving back home, after visiting one of my best friends for the weekend, and realized that I was happy.  My internship is going alright (as far as I know).  I've got a good roommate.  My health is in order, and I've lost 10 pounds in about a month because I've started eating healthy and exercising.  I've found a cool place for atheists to hang out in Indianapolis.  I'm volunteering at least once a week at Planned Parenthood.  My parents are in good health, and I'm on good terms with my